In [0]:
import dlt
fullpath = "/Volumes/dlt-catalog/dlt-schema/ext-loc-volume/expect/"

In [0]:
@dlt.table(name = "bronze_stream_data")
def stream_data ():
    df = spark.readStream.format("cloudFiles").option("cloudFiles.format","csv").option("header","true").option("sep",",").load(fullpath)
    return df

In [0]:
checks = {}
checks["check Gender col for null"] = "(Gender is not null)"
checks["check DOB col is int"] = "(CAST(DOB AS INT) IS NOT NULL)"

In [0]:
@dlt.view(name = "bronze_stream_data_view")
@dlt.expect_all_or_drop(checks)
def stream_data_view ():
    df = dlt.readStream("bronze_stream_data")
    return df

In [0]:
@dlt.table(name = "silver_stream_data")
def silver_stream_data():
    df = dlt.readStream("bronze_stream_data_view")
    return df

In [0]:
@dlt.table(name = "gold_stream_data")
@dlt.expect_or_fail("Checking for non corrupt records","(_rescued_data IS NULL)")
def silver_stream_data():
    df = dlt.readStream("silver_stream_data")
    return df